### T5 for text simplification v1

In [ ]:
!pip install transformers -q
!pip install datasets -q
!pip install sentencepiece -q
!pip install accelerate -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 71.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 236.8/236.8 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 111.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 82.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.6/485.6 kB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 25.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 57.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
from torch import cuda
device = 'cuda' if cuda.is_available() else 'cpu'
device

'cuda'

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from transformers import pipeline, set_seed
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datasets import load_dataset, load_metric, Dataset
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, T5ForConditionalGeneration
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt')
import torch
from tqdm import tqdm

import json
import dataclasses

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
dataset_wiki = load_dataset("bogdancazan/wikilarge-text-simplification")
dataset_newsela = load_dataset("bogdancazan/news-not-not-ela-text-simplification")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/bogdancazan___csv/bogdancazan--wikilarge-text-simplification-8e4a7866601ffbfd/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


...

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

Dataset csv downloaded and prepared to /root/.cache/huggingface/datasets/bogdancazan___csv/bogdancazan--news-not-not-ela-text-simplification-cdb0ef56704f4c59/0.0.0/eea64c71ca8b46dd3f537ed218fc9bf495d5707789152eb2764f5c78fa66d59d. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset_newsela

DatasetDict({
    train: Dataset({
        features: ['Normal', 'Simple'],
        num_rows: 102711
    })
    test: Dataset({
        features: ['Normal', 'Simple'],
        num_rows: 1414
    })
})

In [ ]:
num_rows = min(len(dataset_newsela['train']), len(dataset_wiki['train']))

In [ ]:
dataset_wiki = load_dataset('bogdancazan/wikilarge-text-simplification', split=[f'train[:{num_rows}]', 'test', 'validation'])
dataset_newsela = load_dataset('bogdancazan/news-not-not-ela-text-simplification', split=[f'train[:{num_rows}]', 'test'])

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
dataset_wiki

[Dataset({
     features: ['Normal', 'Simple'],
     num_rows: 102711
 }),
 Dataset({
     features: ['Normal', 'Simple'],
     num_rows: 191
 }),
 Dataset({
     features: ['Normal', 'Simple'],
     num_rows: 494
 })]

In [ ]:
model_name = 'facebook/bart-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model_bart = AutoModelForSeq2SeqLM.from_pretrained(model_name)

In [ ]:
BATCH_SIZE = 128

In [ ]:
def convert_examples_to_features_hf(example_batch):
  input_encodings = tokenizer(example_batch['Normal'], max_length=80, truncation=True, padding=True, return_tensors='pt')

  target_encodings = tokenizer(example_batch['Simple'], max_length=80, truncation=True, padding=True, return_tensors='pt')

  return {
      'input_ids' : input_encodings['input_ids'].to(dtype=torch.long),
      'attention_mask': input_encodings['attention_mask'].to(dtype=torch.long),
      'labels': target_encodings['input_ids'].to(dtype=torch.long),
  }

In [ ]:
dataset_wiki

[Dataset({
     features: ['Normal', 'Simple'],
     num_rows: 102711
 }),
 Dataset({
     features: ['Normal', 'Simple'],
     num_rows: 191
 }),
 Dataset({
     features: ['Normal', 'Simple'],
     num_rows: 494
 })]

In [ ]:
dataset_wikilarge_train_pt = dataset_wiki[0].map(convert_examples_to_features_hf, batched=True).remove_columns(["Normal", "Simple"])
dataset_wikilarge_validation_pt = dataset_wiki[2].map(convert_examples_to_features_hf, batched=True).remove_columns(["Normal", "Simple"])
dataset_newsela_train_pt = dataset_newsela[0].map(convert_examples_to_features_hf, batched=True).remove_columns(["Normal", "Simple"])
dataset_newsela_test_pt = dataset_newsela[1].map(convert_examples_to_features_hf, batched=True).remove_columns(["Normal", "Simple"])


Map:   0%|          | 0/102711 [00:00<?, ? examples/s]

Map:   0%|          | 0/494 [00:00<?, ? examples/s]

Map:   0%|          | 0/102711 [00:00<?, ? examples/s]

Map:   0%|          | 0/1414 [00:00<?, ? examples/s]

In [ ]:
dataset_wikilarge_train_pt

Dataset({
    features: ['input_ids', 'attention_mask', 'labels'],
    num_rows: 102711
})

In [ ]:
from transformers import DataCollatorForSeq2Seq, TrainingArguments, Trainer

In [ ]:
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_bart)

In [ ]:
from torch.autograd import Function

class GradientReversalFn(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None

In [ ]:
from torch import nn

class DANNTextSimplificationModel(nn.Module):
    def __init__(self, num_labels, model_name):
        super(DANNTextSimplificationModel, self).__init__()

        self.num_labels = num_labels
        self.model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
        self.transf = nn.Linear(50265, 1)
        # self.dropout = nn.Dropout(config["hidden_dropout_prob"])
        # self.sentiment_classifier = nn.Sequential(
        #     nn.Linear(config["hidden_size"], num_labels),
        #     nn.LogSoftmax(dim=1),
        # )

        self.domain_classifier = nn.Sequential(
            # nn.Linear(32128, 32128//2),
            # nn.LeakyReLU(True),
            # nn.Dropout(p=0.2),
            # nn.Linear(32128//2, 2),
            nn.Linear(79, 2),
            nn.LogSoftmax(dim=1),
        )

    def forward(
          self,
          input_ids=None,
          attention_mask=None,
          decoder_input_ids=None,
          labels=None,
          grl_lambda = 1.0,
          ):

        # print(input_ids.shape, attention_mask.shape, decoder_input_ids.shape, labels.shape)

        outputs = self.model(
          input_ids=input_ids,
          attention_mask=attention_mask,
          decoder_input_ids=decoder_input_ids,
          labels=labels
        )

        pooled_output = outputs[1]
        # pooled_output = self.dropout(pooled_output)

        # print(pooled_output.shape)

        reversed_output = GradientReversalFn.apply(pooled_output, grl_lambda)
        if torch.isinf(reversed_output).any():
          clamp_value = torch.finfo(reversed_output.dtype).max - 1000
          reversed_output = torch.clamp(reversed_output, min=-clamp_value, max=clamp_value)

        transf_output = self.transf(reversed_output).squeeze(-1)     # convert to size (batch_size, seq_size) from (batch_size, seq_size, vocab_size)
        if torch.isinf(transf_output).any():
          clamp_value = torch.finfo(transf_output.dtype).max - 1000
          transf_output = torch.clamp(transf_output, min=-clamp_value, max=clamp_value)


        # sentiment_pred = self.sentiment_classifier(pooled_output)
        domain_pred = self.domain_classifier(transf_output)
        if torch.isinf(domain_pred).any():
          clamp_value = torch.finfo(domain_pred.dtype).max - 1000
          domain_pred = torch.clamp(domain_pred, min=-clamp_value, max=clamp_value)


        return (outputs[0].to(device), outputs[1].to(device)), domain_pred.to(device)

In [ ]:
model_bart = DANNTextSimplificationModel(num_labels=2, model_name=model_name).to(device)

In [ ]:
import math
from enum import Enum
from dataclasses import dataclass
from typing import Dict, List, Optional, Union, Tuple, NamedTuple
import time
import importlib.util
import importlib.metadata as importlib_metadata
from packaging import version
from torch.utils.data import DataLoader, Dataset, RandomSampler, SequentialSampler
import os

def has_length(dataset):
    """
    Checks if the dataset implements __len__() and it doesn't raise an error
    """
    try:
        return len(dataset) is not None
    except TypeError:
        # TypeError: len() of unsized object
        return False

class ExplicitEnum(str, Enum):
    """
    Enum with more explicit error message for missing values.
    """

class DebugOption(ExplicitEnum):
    UNDERFLOW_OVERFLOW = "underflow_overflow"
    TPU_METRICS_DEBUG = "tpu_metrics_debug"

@dataclass
class TrainerState:
    """
    A class containing the [`Trainer`] inner state that will be saved along the model and optimizer when checkpointing
    and passed to the [`TrainerCallback`].

    <Tip>

    In all this class, one step is to be understood as one update step. When using gradient accumulation, one update
    step may require several forward and backward passes: if you use `gradient_accumulation_steps=n`, then one update
    step requires going through *n* batches.

    </Tip>

    Args:
        epoch (`float`, *optional*):
            Only set during training, will represent the epoch the training is at (the decimal part being the
            percentage of the current epoch completed).
        global_step (`int`, *optional*, defaults to 0):
            During training, represents the number of update steps completed.
        max_steps (`int`, *optional*, defaults to 0):
            The number of update steps to do during the current training.
        total_flos (`float`, *optional*, defaults to 0):
            The total number of floating operations done by the model since the beginning of training (stored as floats
            to avoid overflow).
        log_history (`List[Dict[str, float]]`, *optional*):
            The list of logs done since the beginning of training.
        best_metric (`float`, *optional*):
            When tracking the best model, the value of the best metric encountered so far.
        best_model_checkpoint (`str`, *optional*):
            When tracking the best model, the value of the name of the checkpoint for the best model encountered so
            far.
        is_local_process_zero (`bool`, *optional*, defaults to `True`):
            Whether or not this process is the local (e.g., on one machine if training in a distributed fashion on
            several machines) main process.
        is_world_process_zero (`bool`, *optional*, defaults to `True`):
            Whether or not this process is the global main process (when training in a distributed fashion on several
            machines, this is only going to be `True` for one process).
        is_hyper_param_search (`bool`, *optional*, defaults to `False`):
            Whether we are in the process of a hyper parameter search using Trainer.hyperparameter_search. This will
            impact the way data will be logged in TensorBoard.
    """

    epoch: Optional[float] = None
    global_step: int = 0
    max_steps: int = 0
    num_train_epochs: int = 0
    total_flos: float = 0
    log_history: List[Dict[str, float]] = None
    best_metric: Optional[float] = None
    best_model_checkpoint: Optional[str] = None
    is_local_process_zero: bool = True
    is_world_process_zero: bool = True
    is_hyper_param_search: bool = False
    trial_name: str = None
    trial_params: Dict[str, Union[str, float, int, bool]] = None

    def __post_init__(self):
        if self.log_history is None:
            self.log_history = []

    def save_to_json(self, json_path: str):
        """Save the content of this instance in JSON format inside `json_path`."""
        json_string = json.dumps(dataclasses.asdict(self), indent=2, sort_keys=True) + "\n"
        with open(json_path, "w", encoding="utf-8") as f:
            f.write(json_string)

    @classmethod
    def load_from_json(cls, json_path: str):
        """Create an instance from the content of `json_path`."""
        with open(json_path, "r", encoding="utf-8") as f:
            text = f.read()
        return cls(**json.loads(text))


@dataclass
class TrainerControl:
    """
    A class that handles the [`Trainer`] control flow. This class is used by the [`TrainerCallback`] to activate some
    switches in the training loop.

    Args:
        should_training_stop (`bool`, *optional*, defaults to `False`):
            Whether or not the training should be interrupted.

            If `True`, this variable will not be set back to `False`. The training will just stop.
        should_epoch_stop (`bool`, *optional*, defaults to `False`):
            Whether or not the current epoch should be interrupted.

            If `True`, this variable will be set back to `False` at the beginning of the next epoch.
        should_save (`bool`, *optional*, defaults to `False`):
            Whether or not the model should be saved at this step.

            If `True`, this variable will be set back to `False` at the beginning of the next step.
        should_evaluate (`bool`, *optional*, defaults to `False`):
            Whether or not the model should be evaluated at this step.

            If `True`, this variable will be set back to `False` at the beginning of the next step.
        should_log (`bool`, *optional*, defaults to `False`):
            Whether or not the logs should be reported at this step.

            If `True`, this variable will be set back to `False` at the beginning of the next step.
    """

    should_training_stop: bool = False
    should_epoch_stop: bool = False
    should_save: bool = False
    should_evaluate: bool = False
    should_log: bool = False

    def _new_training(self):
        """Internal method that resets the variable for a new training."""
        self.should_training_stop = False

    def _new_epoch(self):
        """Internal method that resets the variable for a new epoch."""
        self.should_epoch_stop = False

    def _new_step(self):
        """Internal method that resets the variable for a new step."""
        self.should_save = False
        self.should_evaluate = False
        self.should_log = False

# TODO: This doesn't work for all packages (`bs4`, `faiss`, etc.) Talk to Sylvain to see how to do with it better.
def _is_package_available(pkg_name: str, return_version: bool = False) -> Union[Tuple[bool, str], bool]:
    # Check we're not importing a "pkg_name" directory somewhere but the actual library by trying to grab the version
    package_exists = importlib.util.find_spec(pkg_name) is not None
    package_version = "N/A"
    if package_exists:
        try:
            package_version = importlib_metadata.version(pkg_name)
            package_exists = True
        except importlib_metadata.PackageNotFoundError:
            package_exists = False
    if return_version:
        return package_exists, package_version
    else:
        return package_exists

_accelerate_available, _accelerate_version = _is_package_available("accelerate", return_version=True)
def is_accelerate_available(min_version: str = None):
    if min_version is not None:
        return _accelerate_available and version.parse(_accelerate_version) >= version.parse(min_version)
    return _accelerate_available

skip_first_batches = None
if is_accelerate_available():
    from accelerate import __version__ as accelerate_version

    if version.parse(accelerate_version) >= version.parse("0.16"):
        from accelerate import skip_first_batches

    from accelerate import Accelerator
    from accelerate.utils import DistributedDataParallelKwargs

def seed_worker(_):
    """
    Helper function to set worker seed during Dataloader initialization.
    """
    worker_seed = torch.initial_seed() % 2**32
    set_seed(worker_seed)

def speed_metrics(split, start_time, num_samples=None, num_steps=None):
    """
    Measure and return speed performance metrics.

    This function requires a time snapshot `start_time` before the operation to be measured starts and this function
    should be run immediately after the operation to be measured has completed.

    Args:
    - split: name to prefix metric (like train, eval, test...)
    - start_time: operation start time
    - num_samples: number of samples processed
    """
    runtime = time.time() - start_time
    result = {f"{split}_runtime": round(runtime, 4)}
    if runtime == 0:
        return result
    if num_samples is not None:
        samples_per_second = num_samples / runtime
        result[f"{split}_samples_per_second"] = round(samples_per_second, 3)
    if num_steps is not None:
        steps_per_second = num_steps / runtime
        result[f"{split}_steps_per_second"] = round(steps_per_second, 3)
    return result

class TrainOutput(NamedTuple):
    global_step: int
    training_loss: float
    metrics: Dict[str, float]

class ShardedDDPOption(ExplicitEnum):
    SIMPLE = "simple"
    ZERO_DP_2 = "zero_dp_2"
    ZERO_DP_3 = "zero_dp_3"
    OFFLOAD = "offload"
    AUTO_WRAP = "auto_wrap"

def is_peft_available():
    return _is_package_available("peft")

In [ ]:
skip_first_batches = None
if is_accelerate_available():
    from accelerate import __version__ as accelerate_version

    if version.parse(accelerate_version) >= version.parse("0.16"):
        from accelerate import skip_first_batches

    from accelerate import Accelerator
    from accelerate.utils import DistributedDataParallelKwargs

class CustomTrainer(Trainer):

  def __init__(self,
        model=None,
        args=None,
        data_collator=None,
        train_dataset=None,
        eval_dataset=None,
        tokenizer=None,
        model_init=None,
        compute_metrics=None,
        callbacks=None,
        optimizers=(None, None),
        preprocess_logits_for_metrics=None,
        target_dataset=None,
      ) -> None:
      super().__init__(model=model, args=args, data_collator=data_collator, train_dataset=train_dataset, eval_dataset=eval_dataset, tokenizer=tokenizer, model_init=model_init, compute_metrics=compute_metrics, callbacks=callbacks, optimizers=optimizers, preprocess_logits_for_metrics=preprocess_logits_for_metrics)
      self.target_dataset = target_dataset
      self.inner_model = model.model

  def get_target_dataloader(self) -> DataLoader:
        """
        Returns the training [`~torch.utils.data.DataLoader`].

        Will use no sampler if `train_dataset` does not implement `__len__`, a random sampler (adapted to distributed
        training if necessary) otherwise.

        Subclass and override this method if you want to inject some custom behavior.
        """
        if self.target_dataset is None:
            raise ValueError("Trainer: training requires a target_dataset.")

        target_dataset = self.target_dataset
        data_collator = self.data_collator

        dataloader_params = {
            "batch_size": self._train_batch_size,
            "collate_fn": data_collator,
            "num_workers": self.args.dataloader_num_workers,
            "pin_memory": self.args.dataloader_pin_memory,
        }

        if not isinstance(target_dataset, torch.utils.data.IterableDataset):
            dataloader_params["sampler"] = self._get_train_sampler()
            dataloader_params["drop_last"] = self.args.dataloader_drop_last
            dataloader_params["worker_init_fn"] = seed_worker

        return self.accelerator.prepare(DataLoader(target_dataset, **dataloader_params))

  def _inner_training_loop(
        self, batch_size=None, args=None, resume_from_checkpoint=None, trial=None, ignore_keys_for_eval=None
    ):
        self.accelerator.free_memory()
        self._train_batch_size = batch_size
        # Data loader and number of training steps
        train_dataloader = self.get_train_dataloader()
        target_dataloader = self.get_target_dataloader()

        # Setting up training control variables:
        # number of training epochs: num_train_epochs
        # number of training steps per epoch: num_update_steps_per_epoch
        # total number of training steps to execute: max_steps
        total_train_batch_size = args.train_batch_size * args.gradient_accumulation_steps * args.world_size

        len_dataloader = None
        if has_length(train_dataloader):
            len_dataloader = len(train_dataloader)
            num_update_steps_per_epoch = len_dataloader // args.gradient_accumulation_steps
            num_update_steps_per_epoch = max(num_update_steps_per_epoch, 1)
            num_examples = self.num_examples(train_dataloader)
            if args.max_steps > 0:
                max_steps = args.max_steps
                num_train_epochs = args.max_steps // num_update_steps_per_epoch + int(
                    args.max_steps % num_update_steps_per_epoch > 0
                )
                # May be slightly incorrect if the last batch in the training dataloader has a smaller size but it's
                # the best we can do.
                num_train_samples = args.max_steps * total_train_batch_size
            else:
                max_steps = math.ceil(args.num_train_epochs * num_update_steps_per_epoch)
                num_train_epochs = math.ceil(args.num_train_epochs)
                num_train_samples = self.num_examples(train_dataloader) * args.num_train_epochs
        elif args.max_steps > 0:  # Rely on max_steps when dataloader does not have a working size
            max_steps = args.max_steps
            # Setting a very large number of epochs so we go as many times as necessary over the iterator.
            num_train_epochs = sys.maxsize
            num_update_steps_per_epoch = max_steps
            num_examples = total_train_batch_size * args.max_steps
            num_train_samples = args.max_steps * total_train_batch_size
        else:
            raise ValueError(
                "args.max_steps must be set to a positive value if dataloader does not have a length, was"
                f" {args.max_steps}"
            )

        # Compute absolute values for logging, eval, and save if given as ratio
        if args.logging_steps and args.logging_steps < 1:
            args.logging_steps = math.ceil(max_steps * args.logging_steps)
        if args.eval_steps and args.eval_steps < 1:
            args.eval_steps = math.ceil(max_steps * args.eval_steps)
        if args.save_steps and args.save_steps < 1:
            args.save_steps = math.ceil(max_steps * args.save_steps)

        if DebugOption.UNDERFLOW_OVERFLOW in self.args.debug:
            if self.args.n_gpu > 1:
                # nn.DataParallel(model) replicates the model, creating new variables and module
                # references registered here no longer work on other gpus, breaking the module
                raise ValueError(
                    "Currently --debug underflow_overflow is not supported under DP. Please use DDP"
                    " (torch.distributed.launch)."
                )
            else:
                debug_overflow = DebugUnderflowOverflow(self.model)  # noqa

        delay_optimizer_creation = (
            self.sharded_ddp is not None
            and self.sharded_ddp != ShardedDDPOption.SIMPLE
            or False
            or self.fsdp is not None
        )

        if self.is_deepspeed_enabled:
            self.optimizer, self.lr_scheduler = deepspeed_init(self, num_training_steps=max_steps)

        if not delay_optimizer_creation:
            self.create_optimizer_and_scheduler(num_training_steps=max_steps)

        self.state = TrainerState()
        self.state.is_hyper_param_search = trial is not None

        # Activate gradient checkpointing if needed
        if args.gradient_checkpointing:
            self.model.gradient_checkpointing_enable()

        model = self._wrap_model(self.model_wrapped)

        if False and resume_from_checkpoint is not None:
            self._load_from_checkpoint(resume_from_checkpoint, model)

        # as the model is wrapped, don't use `accelerator.prepare`
        # this is for unhandled cases such as
        # Fairscale Sharded DDP, FSDP-XLA, SageMaker MP/DP, DataParallel, IPEX
        use_accelerator_prepare = True if model is self.model else False

        if delay_optimizer_creation:
            self.create_optimizer_and_scheduler(num_training_steps=max_steps)

        # prepare using `accelerator` prepare
        if use_accelerator_prepare:
            if hasattr(self.lr_scheduler, "step"):
                if self.use_apex:
                    model = self.accelerator.prepare(self.model)
                else:
                    model, self.optimizer = self.accelerator.prepare(self.model, self.optimizer)
            else:
                # to handle cases wherein we pass "DummyScheduler" such as when it is specified in DeepSpeed config.
                model, self.optimizer, self.lr_scheduler = self.accelerator.prepare(
                    self.model, self.optimizer, self.lr_scheduler
                )

        if self.is_fsdp_enabled:
            self.model = model

        # for the rest of this function `model` is the outside model, whether it was wrapped or not
        if model is not self.model:
            self.model_wrapped = model

        # backward compatibility
        if self.is_deepspeed_enabled:
            self.deepspeed = self.model_wrapped

        # deepspeed ckpt loading
        if resume_from_checkpoint is not None and self.is_deepspeed_enabled:
            deepspeed_load_checkpoint(self.model_wrapped, resume_from_checkpoint)

        # Check if saved optimizer or scheduler states exist
        self._load_optimizer_and_scheduler(resume_from_checkpoint)

        # important: at this point:
        # self.model         is the Transformers Model
        # self.model_wrapped is DDP(Transformers Model), Deepspeed(Transformers Model), etc.

        self.state.epoch = 0
        start_time = time.time()
        epochs_trained = 0
        steps_trained_in_current_epoch = 0
        steps_trained_progress_bar = None

        # Check if continuing training from a checkpoint
        if resume_from_checkpoint is not None and os.path.isfile(
            os.path.join(resume_from_checkpoint, TRAINER_STATE_NAME)
        ):
            self.state = TrainerState.load_from_json(os.path.join(resume_from_checkpoint, TRAINER_STATE_NAME))
            epochs_trained = self.state.global_step // num_update_steps_per_epoch
            if not args.ignore_data_skip:
                steps_trained_in_current_epoch = self.state.global_step % (num_update_steps_per_epoch)
                steps_trained_in_current_epoch *= args.gradient_accumulation_steps
            else:
                steps_trained_in_current_epoch = 0

            if not args.ignore_data_skip:
                if skip_first_batches is None:
                    pass
                else:
                  pass
                if self.is_local_process_zero() and not args.disable_tqdm and skip_first_batches is None:
                    steps_trained_progress_bar = tqdm(total=steps_trained_in_current_epoch)
                    steps_trained_progress_bar.set_description("Skipping the first batches")

        # Update the references
        self.callback_handler.model = self.model
        self.callback_handler.optimizer = self.optimizer
        self.callback_handler.lr_scheduler = self.lr_scheduler
        self.callback_handler.train_dataloader = train_dataloader
        if self.hp_name is not None and self._trial is not None:
            # use self._trial because the SigOpt/Optuna hpo only call `_hp_search_setup(trial)` instead of passing trial
            # parameter to Train when using DDP.
            self.state.trial_name = self.hp_name(self._trial)
        if trial is not None:
            assignments = trial.assignments if self.hp_search_backend == HPSearchBackend.SIGOPT else trial
            self.state.trial_params = hp_params(assignments)
        else:
            self.state.trial_params = None
        # This should be the same if the state has been saved but in case the training arguments changed, it's safer
        # to set this after the load.
        self.state.max_steps = max_steps
        self.state.num_train_epochs = num_train_epochs
        self.state.is_local_process_zero = self.is_local_process_zero()
        self.state.is_world_process_zero = self.is_world_process_zero()

        # tr_loss is a tensor to avoid synchronization of TPUs through .item()
        tr_loss = torch.tensor(0.0).to(args.device)
        # _total_loss_scalar is updated everytime .item() has to be called on tr_loss and stores the sum of all losses
        self._total_loss_scalar = 0.0
        self._globalstep_last_logged = self.state.global_step
        model.zero_grad()

        self.control = self.callback_handler.on_train_begin(args, self.state, self.control)

        # Skip the first epochs_trained epochs to get the random state of the dataloader at the right point.
        if not args.ignore_data_skip:
            for epoch in range(epochs_trained):
                for _ in train_dataloader:
                    break
            for epoch in range(epochs_trained):
                for _ in train_dataloader:
                  break

        total_batched_samples = 0
        for epoch in range(epochs_trained, num_train_epochs):
            epoch_iterator = train_dataloader
            target_epoch_iterator = target_dataloader

            # Reset the past mems state at the beginning of each epoch if necessary.
            if args.past_index >= 0:
                self._past = None

            steps_in_epoch = (
                len(epoch_iterator)
                if len_dataloader is not None
                else args.max_steps * args.gradient_accumulation_steps
            )
            self.control = self.callback_handler.on_epoch_begin(args, self.state, self.control)

            if epoch == epochs_trained and resume_from_checkpoint is not None and steps_trained_in_current_epoch == 0:
                self._load_rng_state(resume_from_checkpoint)

            rng_to_sync = False
            steps_skipped = 0
            if skip_first_batches is not None and steps_trained_in_current_epoch > 0:
                epoch_iterator = skip_first_batches(epoch_iterator, steps_trained_in_current_epoch)
                target_epoch_iterator = skip_first_batches(target_epoch_iterator, steps_trained_in_current_epoch)
                steps_skipped = steps_trained_in_current_epoch
                steps_trained_in_current_epoch = 0
                rng_to_sync = True

            step = -1
            target_epoch_iterator_object = iter(target_epoch_iterator)
            loss_fn_domain_classifier = torch.nn.NLLLoss()
            for step, inputs in enumerate(epoch_iterator):
                p = float(step + epoch * BATCH_SIZE) / (num_train_epochs * BATCH_SIZE)
                grl_lambda = 2. / (1. + np.exp(-10 * p)) - 1
                grl_lambda = torch.tensor(grl_lambda)

                target_inputs = next(target_epoch_iterator_object)
                total_batched_samples += 1
                if rng_to_sync:
                    self._load_rng_state(resume_from_checkpoint)
                    rng_to_sync = False

                # Skip past any already trained steps if resuming training
                if steps_trained_in_current_epoch > 0:
                    steps_trained_in_current_epoch -= 1
                    if steps_trained_progress_bar is not None:
                        steps_trained_progress_bar.update(1)
                    if steps_trained_in_current_epoch == 0:
                        self._load_rng_state(resume_from_checkpoint)
                    continue
                elif steps_trained_progress_bar is not None:
                    steps_trained_progress_bar.close()
                    steps_trained_progress_bar = None

                if step % args.gradient_accumulation_steps == 0:
                    self.control = self.callback_handler.on_step_begin(args, self.state, self.control)

                with self.accelerator.accumulate(model):
                    tr_loss_step = self.training_step(model, inputs, target_inputs, loss_fn_domain_classifier, grl_lambda)

                if (
                    args.logging_nan_inf_filter
                    and not False
                    and (torch.isnan(tr_loss_step) or torch.isinf(tr_loss_step))
                ):
                    # if loss is nan or inf simply add the average of previous logged losses
                    tr_loss += tr_loss / (1 + self.state.global_step - self._globalstep_last_logged)
                else:
                    tr_loss += tr_loss_step

                self.current_flos += float(self.floating_point_ops(inputs))

                # should this be under the accumulate context manager?
                # the `or` condition of `steps_in_epoch <= args.gradient_accumulation_steps` is not covered
                # in accelerate
                if total_batched_samples % args.gradient_accumulation_steps == 0 or (
                    # last step in epoch but step is always smaller than gradient_accumulation_steps
                    steps_in_epoch <= args.gradient_accumulation_steps
                    and (step + 1) == steps_in_epoch
                ):
                    # Gradient clipping
                    if args.max_grad_norm is not None and args.max_grad_norm > 0:
                        # deepspeed does its own clipping

                        if self.do_grad_scaling:
                            # Reduce gradients first for XLA
                            if False:
                                gradients = xm._fetch_gradients(self.optimizer)
                                xm.all_reduce("sum", gradients, scale=1.0 / xm.xrt_world_size())
                            # AMP: gradients need unscaling
                            self.scaler.unscale_(self.optimizer)

                        if False and args.fp16:
                            self.optimizer.clip_master_grads(args.max_grad_norm)
                        elif hasattr(self.optimizer, "clip_grad_norm"):
                            # Some optimizers (like the sharded optimizer) have a specific way to do gradient clipping
                            self.optimizer.clip_grad_norm(args.max_grad_norm)
                        elif hasattr(model, "clip_grad_norm_"):
                            # Some models (like FullyShardedDDP) have a specific way to do gradient clipping
                            model.clip_grad_norm_(args.max_grad_norm)
                        elif self.use_apex:
                            # Revert to normal clipping otherwise, handling Apex or full precision
                            nn.utils.clip_grad_norm_(
                                amp.master_params(self.optimizer),
                                args.max_grad_norm,
                            )
                        else:
                            self.accelerator.clip_grad_norm_(
                                model.parameters(),
                                args.max_grad_norm,
                            )

                    # Optimizer step
                    optimizer_was_run = True
                    if False:
                        if self.do_grad_scaling:
                            self.scaler.step(self.optimizer)
                            self.scaler.update()
                        else:
                            xm.optimizer_step(self.optimizer)
                    elif self.do_grad_scaling:
                        scale_before = self.scaler.get_scale()
                        self.scaler.step(self.optimizer)
                        self.scaler.update()
                        scale_after = self.scaler.get_scale()
                        optimizer_was_run = scale_before <= scale_after
                    else:
                        self.optimizer.step()
                        optimizer_was_run = not self.accelerator.optimizer_step_was_skipped

                    if optimizer_was_run:
                        # Delay optimizer scheduling until metrics are generated
                        if not isinstance(self.lr_scheduler, torch.optim.lr_scheduler.ReduceLROnPlateau):
                            self.lr_scheduler.step()

                    model.zero_grad()
                    self.state.global_step += 1
                    self.state.epoch = epoch + (step + 1 + steps_skipped) / steps_in_epoch
                    self.control = self.callback_handler.on_step_end(args, self.state, self.control)

                    self._maybe_log_save_evaluate(tr_loss, model, trial, epoch, ignore_keys_for_eval)
                else:
                    self.control = self.callback_handler.on_substep_end(args, self.state, self.control)

                if self.control.should_epoch_stop or self.control.should_training_stop:
                    break
            if step < 0:
                self.control.should_training_stop = True

            self.control = self.callback_handler.on_epoch_end(args, self.state, self.control)
            self._maybe_log_save_evaluate(tr_loss, model, trial, epoch, ignore_keys_for_eval)

            if DebugOption.TPU_METRICS_DEBUG in self.args.debug:
                if False:
                    # tpu-comment: Logging debug metrics for PyTorch/XLA (compile, execute times, ops, etc.)
                    xm.master_print(met.metrics_report())
                else:
                    pass
            if self.control.should_training_stop:
                break

        if args.past_index and hasattr(self, "_past"):
            # Clean the state at the end of training
            delattr(self, "_past")

        if args.load_best_model_at_end and self.state.best_model_checkpoint is not None:
            # Wait for everyone to get here so we are sur the model has been saved by process 0.
            if False:
                xm.rendezvous("load_best_model_at_end")
            elif args.parallel_mode == ParallelMode.DISTRIBUTED:
                dist.barrier()
            elif False:
                smp.barrier()

            self._load_best_model()

        # add remaining tr_loss
        self._total_loss_scalar += tr_loss.item()
        train_loss = self._total_loss_scalar / self.state.global_step

        metrics = speed_metrics("train", start_time, num_samples=num_train_samples, num_steps=self.state.max_steps)
        self.store_flos()
        metrics["total_flos"] = self.state.total_flos
        metrics["train_loss"] = train_loss

        self.is_in_train = False

        self._memory_tracker.stop_and_update_metrics(metrics)

        self.log(metrics)

        run_dir = self._get_output_dir(trial)
        checkpoints_sorted = self._sorted_checkpoints(use_mtime=False, output_dir=run_dir)

        # Delete the last checkpoint when save_total_limit=1 if it's different from the best checkpoint and process allowed to save.
        if self.args.should_save and self.state.best_model_checkpoint is not None and self.args.save_total_limit == 1:
            for checkpoint in checkpoints_sorted:
                if checkpoint != self.state.best_model_checkpoint:
                    shutil.rmtree(checkpoint)

        self.control = self.callback_handler.on_train_end(args, self.state, self.control)

        return TrainOutput(self.state.global_step, train_loss, metrics)

  def training_step(self, model, inputs, target_inputs, loss_fn_domain_classifier, grl_lambda):
        """
        Perform a training step on a batch of inputs.

        Subclass and override to inject custom behavior.

        Args:
            model (`nn.Module`):
                The model to train.
            inputs (`Dict[str, Union[torch.Tensor, Any]]`):
                The inputs and targets of the model.

                The dictionary will be unpacked before being fed to the model. Most models expect the targets under the
                argument `labels`. Check your model's documentation for all accepted arguments.

        Return:
            `torch.Tensor`: The tensor with training loss on this batch.
        """
        model.train()
        inputs = self._prepare_inputs(inputs)
        target_inputs = self._prepare_inputs(target_inputs)

        with self.compute_loss_context_manager():
            loss = self.compute_loss(model, inputs, target_inputs, loss_fn_domain_classifier, grl_lambda)

        if self.args.n_gpu > 1:
            loss = loss.mean()  # mean() to average on multi-gpu parallel training

        if self.do_grad_scaling:
            self.scaler.scale(loss).backward()
        elif self.use_apex:
            with amp.scale_loss(loss, self.optimizer) as scaled_loss:
                scaled_loss.backward()
        else:
            self.accelerator.backward(loss)

        return loss.detach() / self.args.gradient_accumulation_steps

  def _save(self, output_dir: Optional[str] = None, state_dict=None):
    # If we are executing this function, we are the process zero, so we don't check for that.
    output_dir = output_dir if output_dir is not None else self.args.output_dir
    os.makedirs(output_dir, exist_ok=True)

    # supported_classes = (PreTrainedModel,) if not is_peft_available() else (PreTrainedModel, PeftModel)
    # Save a trained model and configuration using `save_pretrained()`.
    # They can then be reloaded using `from_pretrained()`
    # if not isinstance(self.inner_model, supported_classes):
    if False:
        if state_dict is None:
            state_dict = self.inner_model.state_dict()

        # if isinstance(unwrap_model(self.inner_model), supported_classes):
        if False:
            unwrap_model(self.inner_model).save_pretrained(
                output_dir, state_dict=state_dict, safe_serialization=self.args.save_safetensors
            )
        else:
            if self.args.save_safetensors:
                safetensors.torch.save_file(state_dict, os.path.join(output_dir, SAFE_WEIGHTS_NAME))
            else:
                torch.save(state_dict, os.path.join(output_dir, WEIGHTS_NAME))
    else:
        self.inner_model.save_pretrained(
            output_dir, state_dict=state_dict, safe_serialization=self.args.save_safetensors
        )

    if self.tokenizer is not None:
        self.tokenizer.save_pretrained(output_dir)

    # Good practice: save your training arguments together with the trained model
    torch.save(self.args, os.path.join(output_dir, "training_args.bin"))


  def save_model(self, output_dir: Optional[str] = None, _internal_call: bool = False):
    """
    Will save the model, so you can reload it using `from_pretrained()`.

    Will only save from the main process.
    """

    if output_dir is None:
        output_dir = self.args.output_dir

    if False:
        self._save_tpu(output_dir)
    elif False:
        # Calling the state_dict needs to be done on the wrapped model and on all processes.
        os.makedirs(output_dir, exist_ok=True)
        state_dict = self.model_wrapped.state_dict()
        if self.args.should_save:
            self._save(output_dir, state_dict=state_dict)
        if IS_SAGEMAKER_MP_POST_1_10:
            # 'user_content.pt' indicates model state_dict saved with smp >= 1.10
            Path(os.path.join(output_dir, "user_content.pt")).touch()
    elif (
        ShardedDDPOption.ZERO_DP_2 in self.args.sharded_ddp
        or ShardedDDPOption.ZERO_DP_3 in self.args.sharded_ddp
        or self.fsdp is not None
        or self.is_fsdp_enabled
    ):
        if self.is_fsdp_enabled:
            os.makedirs(output_dir, exist_ok=True)
            save_fsdp_model(self.accelerator.state.fsdp_plugin, self.accelerator, self.model, output_dir)
        else:
            state_dict = self.model.state_dict()

            if self.args.should_save:
                self._save(output_dir, state_dict=state_dict)
    elif self.is_deepspeed_enabled:
        # this takes care of everything as long as we aren't under zero3
        if self.args.should_save:
            self._save(output_dir)

        if is_deepspeed_zero3_enabled():
            # It's too complicated to try to override different places where the weights dump gets
            # saved, so since under zero3 the file is bogus, simply delete it. The user should
            # either user deepspeed checkpoint to resume or to recover full weights use
            # zero_to_fp32.py stored in the checkpoint.
            if self.args.should_save:
                file = os.path.join(output_dir, WEIGHTS_NAME)
                if os.path.isfile(file):
                    # logger.info(f"deepspeed zero3: removing {file}, see zero_to_fp32.py to recover weights")
                    os.remove(file)

            # now save the real model if stage3_gather_16bit_weights_on_model_save=True
            # if false it will not be saved.
            # This must be called on all ranks
            if not self.model_wrapped.save_16bit_model(output_dir, WEIGHTS_NAME):
                self.inner_model.save_checkpoint(output_dir)

    elif self.args.should_save:
        self._save(output_dir)

    # Push to the Hub when `save_model` is called by the user.
    if self.args.push_to_hub and not _internal_call:
        self.push_to_hub(commit_message="Model save")

  def compute_loss(self, model, inputs, target_inputs, loss_fn_domain_classifier, grl_lambda, return_outputs=False):
    """
    How the loss is computed by Trainer. By default, all models return the loss in the first element.

    Subclass and override for custom behavior.
    """
    if self.label_smoother is not None and "labels" in inputs:
            labels = inputs.pop("labels")
    else:
        labels = None

    input_ids = inputs["input_ids"].to(device, dtype=torch.long)
    attention_mask = inputs["attention_mask"].to(device, dtype=torch.long)
    y = inputs['labels'].to(device, dtype = torch.long)
    decoder_input = y[:, :-1].contiguous()
    decoder_input[decoder_input == -100] = 0
    decoder_input[decoder_input == -1] = 0
    labels = y[:, 1:].clone().detach()
    labels[y[:, 1:] == tokenizer.pad_token_id] = -100

    source_inputs = {
        "input_ids": input_ids.to(dtype=torch.long),
        "attention_mask": attention_mask.to(dtype=torch.long),
        "decoder_input_ids": decoder_input.to(dtype=torch.long),
        "labels": labels.to(dtype=torch.long),
        "grl_lambda": grl_lambda
    }

    for k, v in source_inputs.items():
        source_inputs[k] = v.to(device, dtype=torch.long)

    # print('unde dai: ', source_inputs['input_ids'].shape, source_inputs['attention_mask'].shape, source_inputs['decoder_input_ids'].shape, source_inputs['labels'].shape)

    simplification_pred, source_domain_pred = model(**source_inputs)
    loss_simplification = simplification_pred[0]
    if torch.isinf(loss_simplification).any():
      clamp_value = torch.finfo(loss_simplification.dtype).max - 1000
      loss_simplification = torch.clamp(loss_simplification, min=-clamp_value, max=clamp_value)
    source_domain = torch.zeros(source_domain_pred.shape[0], dtype=torch.long).to(device)
    loss_source_domain = loss_fn_domain_classifier(source_domain_pred, source_domain)
    if torch.isinf(loss_source_domain).any():
      clamp_value = torch.finfo(loss_source_domain.dtype).max - 1000
      loss_source_domain = torch.clamp(loss_source_domain, min=-clamp_value, max=clamp_value)

    target_input_ids = target_inputs["input_ids"].to(device, dtype=torch.long)
    target_attention_mask = target_inputs["attention_mask"].to(device, dtype=torch.long)
    target_y = target_inputs['labels'].to(device, dtype = torch.long)
    if target_y.shape[1] != 80:
        target_y = torch.cat((target_y, torch.zeros(target_y.shape[0], 80-target_y.shape[1]).to(device, dtype=torch.long)), -1)
    # print('target_y', target_y.shape)
    target_decoder_input = target_y[:, :-1].contiguous()
    target_decoder_input[target_decoder_input == -100] = 0
    target_decoder_input[target_decoder_input == -1] = 0
    target_labels = target_y[:, 1:].clone().detach()
    target_labels[target_y[:, 1:] == tokenizer.pad_token_id] = -100

    target_inputs = {
        "input_ids": target_input_ids.to(dtype=torch.long),
        "attention_mask": target_attention_mask.to(dtype=torch.long),
        "decoder_input_ids": target_decoder_input.to(dtype=torch.long),
        "labels": target_labels.to(dtype=torch.long),
        "grl_lambda": grl_lambda
    }

    for k, v in target_inputs.items():
        target_inputs[k] = v.to(device, dtype=torch.long)

    # print('si unde crapa: ', target_inputs['input_ids'].shape, target_inputs['attention_mask'].shape, target_inputs['decoder_input_ids'].shape, target_inputs['labels'].shape)
    # target_inputs['decoder_input_ids'] = None
    _, target_domain_pred = model(**target_inputs)

    # Note that we are not using the sentiment predictions here for updating the weights
    target_domain = torch.ones(target_domain_pred.shape[0], dtype=torch.long).to(device)
    loss_target_domain = loss_fn_domain_classifier(target_domain_pred, target_domain)
    if torch.isinf(loss_target_domain).any():
      clamp_value = torch.finfo(loss_target_domain.dtype).max - 1000
      loss_target_domain = torch.clamp(loss_target_domain, min=-clamp_value, max=clamp_value)

    # Save past state if it exists
    # TODO: this needs to be fixed and made cleaner later.
    if self.args.past_index >= 0:
        self._past = outputs[self.args.past_index]

    if labels is not None:
      loss = loss_simplification + loss_source_domain + loss_target_domain
        # if unwrap_model(model)._get_name() in MODEL_FOR_CAUSAL_LM_MAPPING_NAMES.values():
        #     loss = self.label_smoother(outputs, labels, shift_labels=True)
        # else:
        #     loss = self.label_smoother(outputs, labels)
    else:
        if isinstance(outputs, dict) and "loss" not in outputs:
            raise ValueError(
                "The model did not return a loss from the inputs, only the following keys: "
                f"{','.join(outputs.keys())}. For reference, the inputs it received are {','.join(inputs.keys())}."
            )
        # We don't use .loss here since the model may return tuples instead of ModelOutput.
        # loss = outputs["loss"] if isinstance(outputs, dict) else outputs[0]
        # Combining the loss
        loss = loss_simplification + loss_source_domain + loss_target_domain

    return (loss, outputs) if return_outputs else loss

In [ ]:
training_args = TrainingArguments(
    output_dir='bart-base-wikilarge-newsela-with-domain-adaptation',
    num_train_epochs=20,
    warmup_steps=250,
    per_device_train_batch_size=BATCH_SIZE,
    weight_decay=0.01,
    learning_rate=2e-4,
    optim="adafactor",
    fp16=True
)

In [ ]:
trainer = CustomTrainer(
    model=model_bart,
    args=training_args,
    tokenizer=tokenizer,
    data_collator=seq2seq_data_collator,
    train_dataset=dataset_wikilarge_train_pt,
    target_dataset=dataset_newsela_train_pt,
)

In [ ]:
trainer.train()

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,2.537200
1000,1.902500
1500,1.684600
2000,1.509100
2500,1.420500
3000,1.286300
3500,1.200900
4000,1.150700
4500,1.017300
5000,0.990100


TrainOutput(global_step=16060, training_loss=0.8191675029388846, metrics={'train_runtime': 7069.3076, 'train_samples_per_second': 290.583, 'train_steps_per_second': 2.272, 'total_flos': 0.0, 'train_loss': 0.8191675029388846, 'epoch': 20.0})

In [ ]:
# from huggingface_hub import HfApi
# api = HfApi()

# # Upload all the content from the local folder to your remote Space.
# # By default, files are uploaded at the root of the repo
# api.upload_folder(
#     folder_path="/content/t5-base-wikilarge-newsela-with-domain-adaptation/checkpoint-7500",
#     repo_id="bogdancazan/t5-base-wikilarge-newsela-with-domain-adaptation",
#     repo_type="model",
# )

In [ ]:
# aux_model = AutoModelForSeq2SeqLM.from_pretrained('bogdancazan/t5-base-wikilarge-newsela-with-domain-adaptation').to(device)
# aux_tokenizer = AutoTokenizer.from_pretrained('bogdancazan/t5-base-wikilarge-newsela-with-domain-adaptation')

In [ ]:
normal = 'fitzgerald s relationship with granz was further cemented when he became her manager although it would be nearly a decade before he could record her on one of his many record labels.'

generated_ids = tokenizer(normal, return_tensors="pt").to(device).input_ids

generated_outputs = model_bart.model.generate(generated_ids, min_new_tokens=int(generated_ids.shape[1] * 0.5), max_new_tokens=int(generated_ids.shape[1] * 2), do_sample=False)
generated_text = tokenizer.decode(generated_outputs[0], skip_special_tokens=True)
print(generated_text)

 fitzgerald s relationship with granz became better known when he became her manager. it would be nearly a decade before he could record her on one of his many record labels.


In [ ]:
# normal = 'fitzgerald s relationship with granz was further cemented when he became her manager although it would be nearly a decade before he could record her on one of his many record labels.'

# generated_ids = aux_tokenizer(normal, return_tensors="pt").to(device).input_ids

# generated_outputs = aux_model.generate(generated_ids, min_new_tokens=int(generated_ids.shape[1] * 0.5), max_new_tokens=int(generated_ids.shape[1] * 2), do_sample=False)
# generated_text = aux_tokenizer.decode(generated_outputs[0], skip_special_tokens=True)
# print(generated_text)

granz granz granz granz granz granz granz granz granz granz granz granz granz granz granz granz granz granz granz granz granz granz granz granz granz granz granz granz granz granz


In [ ]:
model_bart.model.push_to_hub('bart-base-wikilarge-newsela-with-domain-adaptation')
tokenizer.push_to_hub('bart-base-wikilarge-newsela-with-domain-adaptation')

Upload 1 LFS files:   0%|          | 0/1 [00:00<?, ?it/s]

pytorch_model.bin:   0%|          | 0.00/558M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/bogdancazan/bart-base-wikilarge-newsela-with-domain-adaptation/commit/5a2ca960cf5629c8c10c1275d3820016c4cbdda7', commit_message='Upload tokenizer', commit_description='', oid='5a2ca960cf5629c8c10c1275d3820016c4cbdda7', pr_url=None, pr_revision=None, pr_num=None)